# Imports

In [20]:
import torch
import cgd_utils
import numpy as np

# Game Setup

In [21]:
# Game constants
num_evaders = 3
num_exits = 6
points_per_inspector = [2, 2, 2]
num_inspectors = len(points_per_inspector)

bounds = [0] + list(np.cumsum(points_per_inspector))
inspector_ranges = [(bounds[i], bounds[i+1]) for i in range(num_inspectors)]

# Check that each point has exactly one inspector.
assert(sum(points_per_inspector) == num_exits)

# Calculate game payoffs
def calculate_expected_payoffs(evader_player_list, inspector_player_list):
    '''Given lists of evader probabilities and inspector probabilities, compute expected payoffs'''
    # Normalize each evader's probabilities to [0, 1].
    normalized_evader_list = (
        [evader_tensor / torch.norm(evader_tensor, 1) 
         for evader_tensor in evader_player_list])
    
    # Normalize each inspector's probabilities to [0,1].
    normalized_inspector_list = (
        [inspect_tensor / torch.norm(inspect_tensor, 1) 
         for inspect_tensor in inspector_player_list])
    
    inspector_probabilities = 1 - torch.cat(normalized_inspector_list)
    
    # Get evader expected payoffs in-order of evader, which is probability that
    # evader choses a point, that the inspector does not (i.e. complement)
    evader_payoffs = [-torch.dot(evader_tensor, inspector_probabilities) 
                      for evader_tensor in normalized_evader_list]
    
    # Define list of inspector payoffs
    inspector_payoffs = [torch.tensor(0.) 
                         for _ in range(num_inspectors)]
    
    # For inspector, payoff is probability that evader and inspector both chose the same exit.
    for evader_tensor in normalized_evader_list:
        for i, ((start, stop), inspector_tensor) in (
                enumerate(zip(inspector_ranges, normalized_inspector_list))):
            
            inspector_payoffs[i] += -torch.dot(inspector_tensor, evader_tensor[start: stop])
            
    return evader_payoffs, inspector_payoffs
            

In [22]:
num_iterations = 1000
learning_rates = [1] * (num_evaders + num_inspectors)

# Define initial probability-ish tensors for evaders and invaders
evader_player_list = [torch.tensor(np.random.uniform(size=(num_exits)), 
                                   requires_grad=True) 
                      for _ in range(num_evaders)]

inspector_player_list = [torch.tensor(np.random.uniform(size=(num_points)), 
                                      requires_grad=True) 
                          for num_points in points_per_inspector]

# Simulate multiple rounds of game
for i in range(num_iterations):
    if (i % 10 == 0):
        print(i)
    evader_payoffs, inspector_payoffs = calculate_expected_payoffs(evader_player_list, inspector_player_list)
    
    updates, _ = cgd_utils.metamatrix_conjugate_gradient(
        evader_payoffs + inspector_payoffs, 
        evader_player_list + inspector_player_list, 
        lr_list=learning_rates)
    
    
    for player, update in zip(evader_player_list + inspector_player_list, updates):
        player.data.add_(update)

# Look at final normalized probabilities
normalized_evader_list = (
        [evader_tensor / torch.norm(evader_tensor, 1) 
         for evader_tensor in evader_player_list])
normalized_inspector_list = (
        [inspect_tensor / torch.norm(inspect_tensor, 1) 
         for inspect_tensor in inspector_player_list])

print("Evader Probabilities")
for tensor in normalized_evader_list:
    print("   " + str(tensor))

print()
    
print("Inspector Probabilities")
for tensor in normalized_inspector_list:
    print("    " + str(tensor))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
Evader Probabilities
   tensor([0.1079, 0.2258, 0.2393, 0.3021, 0.0597, 0.0653], dtype=torch.float64,
       grad_fn=<DivBackward0>)
   tensor([0.3054, 0.2688, 0.1464, 0.1141, 0.1440, 0.0214], dtype=torch.float64,
       grad_fn=<DivBackward0>)
   tensor([0.1958, 0.1144, 0.2329, 0.2023, 0.0688, 0.1858], dtype=torch.float64,
       grad_fn=<DivBackward0>)

Inspector Probabilities
    tensor([0.5000, 0.5000], dtype=torch.float64, grad_fn=<DivBackward0>)
    tensor([0.5000, 0.5000], dtype=torch.float64, grad_fn=<DivBackward0>)
    tensor([0.5000, 0.5000], dtype=torch.float64, grad_fn=<DivBackward0>)


# Simulation with Initial Nash Equilbrium Weights

In [25]:
num_iterations = 1000
learning_rates = [1] * (num_evaders + num_inspectors)

# Define initial probability-ish tensors for evaders and invaders
evader_player_list = [torch.tensor([1/6] * num_exits, 
                                   requires_grad=True) 
                      for _ in range(num_evaders)]

inspector_player_list = [torch.tensor([1/2] * num_points, 
                                      requires_grad=True) 
                          for num_points in points_per_inspector]

# Simulate multiple rounds of game
for i in range(num_iterations):
    if (i % 10 == 0):
        print(i)
    evader_payoffs, inspector_payoffs = calculate_expected_payoffs(evader_player_list, inspector_player_list)
    
    updates, _ = cgd_utils.metamatrix_conjugate_gradient(
        evader_payoffs + inspector_payoffs, 
        evader_player_list + inspector_player_list, 
        lr_list=learning_rates)
    
    
    for player, update in zip(evader_player_list + inspector_player_list, updates):
        player.data.add_(update)

# Look at final normalized probabilities
normalized_evader_list = (
        [evader_tensor / torch.norm(evader_tensor, 1) 
         for evader_tensor in evader_player_list])
normalized_inspector_list = (
        [inspect_tensor / torch.norm(inspect_tensor, 1) 
         for inspect_tensor in inspector_player_list])

print("Evader Probabilities")
for tensor in normalized_evader_list:
    print("   " + str(tensor))

print()
    
print("Inspector Probabilities")
for tensor in normalized_inspector_list:
    print("    " + str(tensor))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
Evader Probabilities
   tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
       grad_fn=<DivBackward0>)
   tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
       grad_fn=<DivBackward0>)
   tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
       grad_fn=<DivBackward0>)

Inspector Probabilities
    tensor([0.5000, 0.5000], grad_fn=<DivBackward0>)
    tensor([0.5000, 0.5000], grad_fn=<DivBackward0>)
    tensor([0.5000, 0.5000], grad_fn=<DivBackward0>)
